# [03] COVID Image Classification: ResNet 구조 만들기

본 실습에서는 이미지 분류에서 널리 알려진 ResNet을 구현해보겠습니다.


(논문: https://www.cv-foundation.org/openaccess/content_cvpr_2016/papers/He_Deep_Residual_Learning_CVPR_2016_paper.pdf)

아래는 Residual Connection입니다. 아래의 그림은 논문에서 발췌하였습니다.

![ResNet](./imgs/residual_connection.png)

ResNet은 크게 `Basic_Block`과 `Bottleneck_Block` 두 가지 구조를 통해 각각 구성됩니다.

첫번째로, `Basic_Block` 구조입니다. 아래는 구조에 대한 그림입니다.

![Basic_Block](./imgs/Basic_Block.png)

`[3x3 Conv, BatchNorm, ReLU]`3가지 층의 구성이 두번 반복되는 것이 Basic block입니다.

이를 토대로 ResNet9 **(18-layer에서 x2 부분을 x1로 바꿈)** 구조를 만들 수 있습니다.

![ResNet18](./imgs/ResNet18.png)

아래의 코드는 파이토치에서 제공하는 official 코드를 조금 고친 것입니다.

먼저 ResNet18에 대한 큰 틀의 class를 구현한 뒤, `Basic Block` class를 구현해 사용해보겠습니다.

In [1]:
import torch
from torch import nn

In [2]:
class ResNet(nn.Module):
    def __init__(self, block, num_classes=2):
        super(ResNet, self).__init__()
        
        self.conv1 = nn.Conv2d(3, ??????, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(??????,)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = block(??????,, ??????,)
        self.layer2 = block(??????,, ??????,, stride=2)
        self.layer3 = block(??????,, ??????,, stride=2)
        self.layer4 = block(??????,, ??????,, stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(??????,, num_classes)

        # initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = ??????
        x = torch.flatten(x, 1)
        ??????

        return x

In [7]:
class BasicBlock(nn.Module):
    def __init__(self, inplanes, outplanes, stride=1):
        super(BasicBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(??????)
        self.bn1 = nn.BatchNorm2d(??????)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(??????)
        self.bn2 = nn.BatchNorm2d(??????)
        
        self.downsample_conv = nn.Conv2d(??????)
        self.downsample_bn = nn.BatchNorm2d(??????)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        identity = self.downsample_conv(x)
        identity = self.downsample_bn(identity)

        out += identity
        out = self.relu(out)

        return out

In [8]:
net = ResNet(BasicBlock)
net

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (downsample_conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (downsample_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer2): BasicBlock(
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True,

두번째로, `Bottleneck Block`입니다.

![Bottleneck](./imgs/bottleneck.png)

In [9]:
class BottleneckBlock(nn.Module):
    def __init__(self, inplanes, outplanes, stride=1):
        super(BottleneckBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(??????)
        self.bn1 = nn.BatchNorm2d(??????)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(??????)
        self.bn2 = nn.BatchNorm2d(??????)
        self.conv3 = nn.Conv2d(??????)
        self.bn3 = nn.BatchNorm2d(??????)
        
        self.downsample_conv = nn.Conv2d(??????)
        self.downsample_bn = nn.BatchNorm2d(??????)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        identity = self.downsample_conv(x)
        identity = self.downsample_bn(identity)

        out += identity
        out = self.relu(out)

        return out

In [10]:
net = ResNet(BottleneckBlock)
net

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): BottleneckBlock(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (con3): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (downsample_conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (downsample_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer2):